<a href="https://colab.research.google.com/github/Kwon-hayoung/news-intelligence-bot/blob/main/notebooks/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install llama-cpp-python huggingface_hub gradio langchain langchain-core langchain-community

In [8]:
from huggingface_hub import login
login()   # 여기 토큰 입력

In [9]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="HaGPT/news-intelligence-chatbot",
    filename="meta-llama-3.1-8b.Q4_K_M.gguf",
    force_download=False
)

model_path

'/root/.cache/huggingface/hub/models--HaGPT--news-intelligence-chatbot/snapshots/517e93fd31f4b1c052e206be34fc2b1aa686537b/meta-llama-3.1-8b.Q4_K_M.gguf'

In [10]:
from llama_cpp import Llama

llm = Llama(
    model_path=model_path,
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=35,   # T4 기준 안전
    verbose=False
)

llama_context: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


GRAIDO CHAT-BOT

In [15]:
import gradio as gr
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.memory import ConversationBufferMemory

# -------------------------------
# 1) 모델 다운로드
# -------------------------------
model_path = hf_hub_download(
    repo_id="HaGPT/news-intelligence-chatbot",
    filename="meta-llama-3.1-8b.Q4_K_M.gguf",
)

# -------------------------------
# 2) 모델 로드
# -------------------------------
llm = Llama(
    model_path=model_path,
    n_ctx=4096,
    chat_format="llama-3",
    temparature=0.3,
)

# -------------------------------
# 3) 메모리
# -------------------------------
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True
)

# -------------------------------
# 4) 챗 함수
# -------------------------------
def chat_fn(message, history):

    # LangChain Memory 로 현재까지 대화 불러오기
    past = memory.load_memory_variables({})["history"]

    messages = []
    for m in past:
        role = "assistant" if m.type == "ai" else "user"
        messages.append({"role": role, "content": m.content})

    # 새 메시지 추가
    messages.append({"role": "user", "content": message})

    # 모델 응답 생성
    result = llm.create_chat_completion(
        messages=messages,
        max_tokens=512,
    )
    bot_reply = result["choices"][0]["message"]["content"]

    # Memory에 저장
    memory.save_context({"input": message}, {"output": bot_reply})

    return bot_reply

# -------------------------------
# 5) Gradio UI
# -------------------------------
demo = gr.ChatInterface(chat_fn)
demo.launch()

llama_model_loader: loaded meta data with 31 key-value pairs and 292 tensors from /root/.cache/huggingface/hub/models--HaGPT--news-intelligence-chatbot/snapshots/517e93fd31f4b1c052e206be34fc2b1aa686537b/meta-llama-3.1-8b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Unsloth_Gguf_A8Xvyz53
llama_model_loader: - kv   3:                       general.quantized_by str              = Unsloth
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - kv   5:                           general.repo_url str              = https://huggingface.co/unsloth
llama_model_

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://28bedef3bd5c7e0fc9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
